# Box varianza 
**Funziona solo con satpy dalla 0.41 in poi, quando è stato inserito nel lettore modis L2 il dataset mod05**

In [ ]:
from glob import glob
import satpy
import xarray as xr
from pyresample.geometry import AreaDefinition, create_area_def, SwathDefinition
from datetime import datetime
#from mast import utils

import hvplot.xarray

## Predisposizione per lettura immagini su disco con satpy

In [ ]:
dir_locale = True
#scene = glob('/home/luca/mounting_point/osvaldo_data/MODIS/MOD05_L2/2017/**/*.hdf',recursive=True)
scene = glob('/home/luca/WORK/DATA/MODIS/2020/**/*.hdf',recursive=True)

In [ ]:
len(scene)

In [ ]:
scene.sort()
#scene

In [ ]:
area_id = 'thule'
description = 'Thule ice box'
proj_id = 'thule'
center = (-67.8,76.37)
#center = (-68,76)
radius = (1000,1000)
shape = (19,19)
resolution =1000

proj_dict = {'proj': 'tmerc', 'lat_0': center[1], 'lon_0': center[0] , 'lat_ts':center[1], 'a': 6371228.0, 'units': 'm'}

center = xr.DataArray([center[0], center[1]], attrs={"units": "degrees"})
area_def = AreaDefinition.from_area_of_interest(area_id, proj_dict, shape, center, resolution)

datasets=[]

In [ ]:
def run_cycle(element):
    try:
        #jj=0
        scn = satpy.Scene([element], reader='modis_l2')
        #jj=1
        scn.load(['water_vapor_infrared'])
        local_scn = scn.resample(area_def, nprocs=16, radius_of_influence=10000)
        local_scn_xr = local_scn.to_xarray_dataset()
        local_scn_xr = local_scn_xr.expand_dims(time=[local_scn_xr.start_time])
        datasets.append(local_scn_xr)
    except Exception as error:
        print("An exception occurred:", error)
            #print(ValueError)
            #pass

## Lettura immagini con satpy

### Ciclo sequenziale

In [ ]:
%%time

ii = 0
for element in scene:
    ii = ii+1
    if dir_locale:
        if ii> 2040:
            print(ii)
    elif dir_locale==False:
        if ii> 1015:
            print(ii)
        
    run_cycle(element)

### Ciclo parallelizzato

### Visualizzazione e concatenazione datasets (perché vanno concatenati? Non lo sono già dopo l'append nella fun run_cycle?)

In [ ]:
print(len(datasets))
array=xr.concat(datasets,dim='time')
array

In [ ]:
variance = array.water_vapor_infrared.var(skipna=True)

In [ ]:
variance.values

In [ ]:
array.water_vapor_infrared.mean().values

In [ ]:
array_var_single = array.water_vapor_infrared.var(dim=['x','y'], skipna=True)

In [ ]:
array_mean_single = array.water_vapor_infrared.mean(dim=['x','y'], skipna=True)

In [ ]:
del array.water_vapor_infrared.attrs['area']
del array.water_vapor_infrared.attrs['_satpy_id']
del array.water_vapor_infrared.attrs['start_time']
del array.water_vapor_infrared.attrs['end_time']

In [ ]:
array.water_vapor_infrared

## Scrittura file netcdf

In [ ]:
array_mean_single.drop('crs').to_netcdf('./Box_Variance/2017_P1_mean_single_19x19.nc')

In [ ]:
array_var_single.drop('crs').to_netcdf('./Box_Variance/2017_P1_var_single_19x19.nc')

In [ ]:
del array.attrs['area']
del array.attrs['_satpy_id']
del array.attrs['start_time']
del array.attrs['end_time']

In [ ]:
array.drop('crs').to_netcdf('./Box_Variance/2017_array_19x19.nc')

## Plot variabili da file

In [ ]:
var5_2017 = xr.open_dataset('./Box_Variance/2017_P1_var_single_5x5.nc')
var19_2017 = xr.open_dataset('./Box_Variance/2017_P1_var_single_19x19.nc')
mean5_2017 = xr.open_dataset('./Box_Variance/2017_P1_mean_single_5x5.nc')
mean19_2017 = xr.open_dataset('./Box_Variance/2017_P1_mean_single_19x19.nc')

In [ ]:
var5_2019 = xr.open_dataset('./Box_Variance/2019_P1_var_single_5x5.nc')
var19_2019 = xr.open_dataset('./Box_Variance/2019_P1_var_single_19x19.nc')
mean5_2019 = xr.open_dataset('./Box_Variance/2019_P1_mean_single_5x5.nc')
mean19_2019 = xr.open_dataset('./Box_Variance/2019_P1_mean_single_19x19.nc')

In [ ]:
var5_2020 = xr.open_dataset('./Box_Variance/2020_P1_var_single_5x5.nc')
var19_2020 = xr.open_dataset('./Box_Variance/2020_P1_var_single_19x19.nc')
mean5_2020 = xr.open_dataset('./Box_Variance/2020_P1_mean_single_5x5.nc')
mean19_2020 = xr.open_dataset('./Box_Variance/2020_P1_mean_single_19x19.nc')

In [ ]:
ylabel='IWV box variance  [mm]'
(var19_2017.hvplot(ylabel=ylabel, label='2017 19x19',legend='top_left') 
 * var5_2017.hvplot(label='2017   5x5',legend='top_left')
).opts(title='MODIS IWV product box variance near Thule            box center:  P1 = (76.37,-67.8)')

In [ ]:
ylabel='IWV box variance  [mm]'
(var19_2019.hvplot(ylabel=ylabel, label='2019 19x19',legend='top_left') 
 * var5_2019.hvplot(label='2019   5x5',legend='top_left')
).opts(title='MODIS IWV product box variance near Thule            box center:  P1 = (76.37,-67.8)')

In [ ]:
ylabel='IWV box variance  [mm]'
(var19_2020.hvplot(ylabel=ylabel, label='2020 19x19',legend='top_left') 
 * var5_2020.hvplot(label='2020   5x5',legend='top_left')
).opts(title='MODIS IWV product box variance near Thule            box center:  P1 = (76.37,-67.8)')

In [ ]:
ylabel='IWV box average   [mm]'
(mean19_2017.hvplot(ylabel=ylabel, label='2017 19x19',legend='top_left') 
 * mean5_2017.hvplot(label='2017   5x5',legend='top_left')
).opts(title='MODIS IWV product box average near Thule            box center:  P1 = (76.37,-67.8)') 

In [ ]:
ylabel='IWV box average   [mm]'
(mean19_2019.hvplot(ylabel=ylabel, label='2019 19x19',legend='top_left') 
 * mean5_2019.hvplot(label='2019   5x5',legend='top_left')
).opts(title='MODIS IWV product box average near Thule            box center:  P1 = (76.37,-67.8)') 

In [ ]:
ylabel='IWV box average   [mm]'
(mean19_2020.hvplot(ylabel=ylabel, label='2020 19x19',legend='top_left') 
 * mean5_2020.hvplot(label='2020   5x5',legend='top_left')
).opts(title='MODIS IWV product box average near Thule            box center:  P1 = (76.37,-67.8)') 